In [ ]:
import pandas as pd
import numpy as np
import qgridnext as qgrid 
from IPython.display import display, clear_output
import ipywidgets as widgets

# Step 1: Create a DataFrame with Multi-Index Columns
data = np.random.randn(4, 8)

columns = pd.MultiIndex.from_product(
    [['Category1', 'Category2'], ['SubCat1', 'SubCat2'], ['Detail1', 'Detail2']],
    names=['Category', 'SubCategory', 'Detail']
)

df = pd.DataFrame(data, index=['Row1', 'Row2', 'Row3', 'Row4'], columns=columns)
# Display the original DataFrame
print("Original DataFrame with Multi-Index Columns:")
display(df)
print("Original DataFrame Index:", df.index.tolist())
print("Original DataFrame Columns:", df.columns.tolist())

# Step 2: Convert Multi-Index Columns to Rows for qgrid
# Extracting the level values and creating a DataFrame for each level
levels = df.columns.names
level_values = [df.columns.get_level_values(i) for i in range(len(levels))]

# Create a DataFrame for the levels
level_dfs = []
for i, level_value in enumerate(level_values):
    level_df = pd.DataFrame([level_value], columns=df.columns)
    level_df.index = [levels[i]]
    level_dfs.append(level_df)

# Create a DataFrame for the values
value_df = pd.DataFrame(data, columns=df.columns, index=df.index)

# Concatenate the levels and values DataFrames
combined_df = pd.concat(level_dfs + [value_df])

# Transpose the DataFrame to fit qgrid format
combined_df = combined_df.T

# Print the transposed DataFrame's index and columns for clarity
print("Transposed DataFrame Index:", combined_df.index.tolist())
print("Transposed DataFrame Columns:", combined_df.columns.tolist())

# Step 3: Ensure unique column names with modifications to avoid duplications
# Adjusting column names to prepend "_" for MultiIndex names
new_columns = []
for i, col in enumerate(combined_df.columns):
    if col in levels:  # Check if the column name is in the original MultiIndex names
        new_columns.append(f'_{col}')  # Prepend "_" to the original name
    else:
        new_columns.append(f'{col}')  # Use the modified name with suffix for unique columns

combined_df.columns = new_columns

# Adjusting row names to append "_" for MultiIndex names
# This is done during the creation of level_dfs
level_dfs_adjusted = []
for i, level_df in enumerate(level_dfs):
    adjusted_index = [f'{index}_' for index in level_df.index]  # Append "_" to each index name
    level_df.index = adjusted_index
    level_dfs_adjusted.append(level_df)

# Recreate the combined DataFrame with adjusted level DataFrames
combined_df_adjusted = pd.concat(level_dfs_adjusted + [value_df]).T

# Now, combined_df_adjusted has "_" prepended to column names that are part of the original MultiIndex
# and "_" appended to row names that are part of the original MultiIndex

# Display the DataFrame with separated levels
print("Combined DataFrame for qgrid:")
display(combined_df)
print("Combined DataFrame Index after adjustments:", combined_df.index.tolist())
print("Combined DataFrame Columns after adjustments:", combined_df.columns.tolist())


# Step 1: Identify columns to keep (those without an underscore)
columns_to_keep = [col for col in combined_df.columns if '_' not in col]

# Step 2: Filter out unnecessary columns
df_cleaned = combined_df[columns_to_keep]

# Step 4: Create qgrid widget and output area for displaying filtered DataFrame
qgrid_widget = qgrid.show_grid(df_cleaned, show_toolbar=True)
output_area = widgets.Output()

# Event handler to update the displayed DataFrame whenever changes are made in qgrid
def on_filter_change(event, widget):
    with output_area:
        clear_output(wait=True)
        filtered_df = widget.get_changed_df().T
        display(filtered_df)

# Attach the event handler to the qgrid widget
qgrid_widget.on('filter_changed', on_filter_change)

# Step 5: Display the qgrid widget and the output area
display(qgrid_widget)
display(output_area)

In [ ]:
import pandas as pd
import numpy as np
import qgridnext as qgrid
import json
from IPython.display import display, clear_output
import ipywidgets as widgets

class MultiIndexDataFrameToQGrid:
    def __init__(self, df):
        self.df = df
        self.combined_df = None
        self.df_cleaned = None
        self.qgrid_widget = None
        self.output_area = widgets.Output()

    def transpose_and_prepare_df(self):
        # Extracting the level values and creating a DataFrame for each level
        levels = self.df.columns.names
        level_values = [self.df.columns.get_level_values(i) for i in range(len(levels))]

        # Create a DataFrame for the levels
        level_dfs = []
        for i, level_value in enumerate(level_values):
            level_df = pd.DataFrame([level_value], columns=self.df.columns)
            level_df.index = [levels[i]]
            level_dfs.append(level_df)

        # Create a DataFrame for the values
        value_df = pd.DataFrame(self.df.values, columns=self.df.columns, index=self.df.index)

        # Concatenate the levels and values DataFrames and transpose
        self.combined_df = pd.concat(level_dfs + [value_df]).T

        # Adjusting column and row names
        new_columns = [col if col not in levels else f'_{col}' for col in self.combined_df.columns]
        self.combined_df.columns = new_columns

        # Filter out unnecessary columns
        columns_to_keep = [col for col in self.combined_df.columns if '_' not in col]
        self.df_cleaned = self.combined_df[columns_to_keep]

    def display_in_qgrid(self):
        # Create qgrid widget for displaying filtered DataFrame
        self.qgrid_widget = qgrid.show_grid(self.df_cleaned, show_toolbar=True)

        # Attach event handler to the qgrid widget
        self.qgrid_widget.on('filter_changed', self.on_filter_change)

        # Display the qgrid widget and the output area
        display(self.qgrid_widget)
        display(self.output_area)

    def on_filter_change(self, event, widget):
        with self.output_area:
            clear_output(wait=True)
            filtered_df = widget.get_changed_df().T
            display(filtered_df)

    
    def write_dataframe_to_json_with_multiindex(self, json_file_path):
        df = self.df
        # Prepare data to include MultiIndex column structure
        data = {
            'data': df.to_json(orient='split', indent=4),
            'columns': df.columns.tolist()
        }
        
        # Serialize the structure to JSON file
        with open(json_file_path, 'w') as file:
            json.dump(data, file, indent=4)
        
    def read_json_as_dataframe_with_multiindex(self, json_file_path):
        # Load the structure from JSON file
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        
        # Reconstruct DataFrame from 'data' part
        df_data = pd.read_json(data['data'], orient='split')
        
        # Reconstruct MultiIndex for columns
        multiindex_columns = pd.MultiIndex.from_tuples(data['columns'])
        df_data.columns = multiindex_columns
        
        return df_data

    

In [ ]:
# Example usage
data = np.random.randn(4, 8).astype(int)
columns = pd.MultiIndex.from_product([['Creatures', 'Spells'], ['Mage', 'Warrior'], ['Input', 'Output']], names=['Category', 'SubCategory', 'Detail'])
df = pd.DataFrame(data, index=['DeckA', 'DeckB', 'DeckC', 'DeckD'], columns=columns)

# Initialize the class with the DataFrame
converter = MultiIndexDataFrameToQGrid(df)

#converter.write_dataframe_to_json_with_multiindex("dataframe.json")
converter.read_json_as_dataframe_with_multiindex("dataframe.json")

# Prepare and transpose the DataFrame
converter.transpose_and_prepare_df()

# Display the DataFrame in qgrid
converter.display_in_qgrid()



In [ ]:
import gspread  
from oauth2client.service_account import ServiceAccountCredentials  
  
# Define the scope and credentials  
scope = ['https://www.googleapis.com/auth/spreadsheets']  
credentials = ServiceAccountCredentials.from_json_keyfile_name('precise-armor-426813-v6-3c3c5c85f7df.json', scope)  
  
# Authorize the client using the credentials  
client = gspread.authorize(credentials)  
  
# Open the Google Sheet by its key  
sheet = client.open_by_key('1RSo-kmJVzam-lVXOsA8N_fKBDDhjyqXFRGjqXkZ_rUQ')  
  
# Select the first worksheet  
worksheet = sheet.worksheet("Card Database")  
  
# Get all values from the worksheet  
values = worksheet.get_all_values()  

print(values)
